In [35]:
import gc
import glob
import os
import re
import sys
sys.path.append("../")
sys.path.append("../../")
from src.make_data import effnet_data as  effnet_data
import utils
from src.model import custom_metric
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pydicom as dicom
import torch
import torchvision as tv
from sklearn.model_selection import GroupKFold
from torch.cuda.amp import GradScaler, autocast
from torchvision.models.feature_extraction import create_feature_extractor
from tqdm import tqdm
import warnings
import joblib
import wandb
from pathlib import Path

warnings.simplefilter('ignore')
# SET CONFIG Effnet

cfg = utils.load_yaml(Path("../../config/config.yaml"))
#DATA PATH
RSNA_2022_PATH = cfg["data"]["RSNA_2022_PATH"]
TRAIN_IMAGES_PATH = f'{RSNA_2022_PATH}/train_images'
TEST_IMAGES_PATH = f'{RSNA_2022_PATH}/test_images'
EFFNET_CHECKPOINTS_PATH = cfg["data"]["EFFNET_CHECKPOINTS_PATH"]
METADATA_PATH = cfg["data"]["METADATA_PATH"]

WEIGHTS = tv.models.efficientnet.EfficientNet_V2_S_Weights.DEFAULT
N_FOLDS = 3
#Read csv data for slicing
df_train = pd.read_csv(f'{RSNA_2022_PATH}/train.csv')
df_train_slices = pd.read_csv(f'{METADATA_PATH}/train_segmented.csv')
df_test = pd.read_csv(f'{RSNA_2022_PATH}/test.csv')
df_train_box = pd.read_csv(f"{RSNA_2022_PATH}/cropped_2d_labels.csv")

#df_train, df_train_slices, df_test, df_test_slices = effnet_data.preprocess( df_train=df_train,
#    df_train_slices=df_train_slices,
#    df_train_box=df_train_box,
#    df_test=df_test,
#    TEST_IMAGES_PATH=TEST_IMAGES_PATH,
#    N_FOLDS=N_FOLDS,
#)
df_train,df_train_slices,df_test,df_test_slices = effnet_data.preprocess(df_train = df_train,df_train_slices=df_train_slices,df_test=df_test,TEST_IMAGES_PATH=TEST_IMAGES_PATH,N_FOLDS=N_FOLDS)
#PreProcess and Effnetdata
data_path = Path("/home/jumpei.uchida/develop/kaggle_1080ti_1_2/rsna-2022-cervical-spine-fracture-detection/fold0")

In [36]:

df = pd.read_csv("/home/jumpei.uchida/develop/kaggle_1080ti_1_2/rsna-2022-cervical-spine-fracture-detection/effnet/src/saved_model/effnet/effnet_third_eval_prediction.csv")

In [37]:
df.tail()

,Unnamed: 0,StudyInstanceUID,Slice,new_slice,patient_overall,split,pred
163612,54459,1.2.826.0.1.3680043.9940,255,255,0.0,2.0,0.009415
163613,54460,1.2.826.0.1.3680043.9940,256,256,0.0,2.0,0.007904
163614,54461,1.2.826.0.1.3680043.9940,257,257,0.0,2.0,0.005980
163615,54462,1.2.826.0.1.3680043.9940,258,258,0.0,2.0,0.004364
163616,54463,1.2.826.0.1.3680043.9940,259,259,0.0,2.0,0.003004


In [38]:

def resize_depth(images: np.ndarray, depth, depth_range, enable_depth_resized_with_cv2):
    assert images.ndim >= 3  # (..., depth, h/w, w/h)

    if depth_range is not None:
        assert len(depth_range) == 2
        start_idx, end_idx = np.quantile(np.arange(images.shape[-3]), depth_range).astype(int)
        images = images[..., start_idx:end_idx, :, :]

    if depth is None:
        return images

    #if images.shape[-3] < depth:
    #    warnings.warn("images.shape[-3] < given depth", UserWarning)

    if enable_depth_resized_with_cv2:
        images = images.swapaxes(-3, -2)
        *left_shapes, images_height, images_depth, images_width = images.shape
        images = images.reshape((-1, images_depth, images_width))
        images = np.stack([
            cv2.resize(image, (images_width, depth), interpolation=cv2.INTER_AREA)
            for image in images
        ], axis=0)
        images = images.reshape((*left_shapes, images_height, depth, images_width))
        images = images.swapaxes(-3, -2)
        return images
    else:
        indices = np.quantile(
            np.arange(images.shape[-3]), np.linspace(0, 1, depth)
        ).astype(int)
        return images[..., indices, :, :]

def transforms(temp,custom = True):
    assert temp.ndim == 4
    c_list = []
    for c in range(7):
        temp_list = []
        for c1 in temp[c]:
            temp_list.append(np.sum(c1).astype("float64"))
        temp_list /= max(temp_list)
        c_list.append(temp_list)
    c_array = np.array(c_list)
    if custom:
        for num_i,sum_num in enumerate(np.sum(c_array,axis = 0)):
            if sum_num > 1.0:
                c_array[:,num_i] /= sum_num
    
    return c_array

def resize_origin(uid,i,flag_df):
    vert_cols = [f'C{i}_effnet_vert' for i in range(1, 8)]
    slice = uid_to_slice_map[uid]
    temp = np.load(data_path / f"{uid}.npz",allow_pickle=True)["arr_0"]
    temp = resize_depth(temp,depth = slice,depth_range = None,enable_depth_resized_with_cv2=True)
    temp = transforms(temp)
    temp = np.nan_to_num(temp)
    temp = pd.DataFrame(temp.T,columns = vert_cols)
    temp["StudyInstanceUID"] = uid
    if flag_df.query("StudyInstanceUID == @uid")["is_reversed"].values[0] == 0:
        temp["Slice"] = [i for i in range(1,slice+1)]
    else:
        temp["Slice"] = list(reversed([i for i in range(1,slice+1)]))
    return temp,i

def get_dicom_paths(dicom_dir_path: Path):
    dicom_paths = sorted(
        dicom_dir_path.glob("*"),
        key=lambda p: int(p.name.split(".")[0])
    )
    if (
        dicom.dcmread(dicom_paths[0]).get("ImagePositionPatient")[2]
        >
        dicom.dcmread(dicom_paths[-1]).get("ImagePositionPatient")[2]
    ):
        return dicom_paths[::-1]
    return dicom_paths
def make_path_list(path,i):
    flag = 1
    temp = get_dicom_paths(path)
    if temp[0].parts[-1] == "1.dcm":
        flag =0
    uid = temp[0].parts[-2]
    return [uid,flag],i


df_eval_pred = df.copy()
uid_to_slice_map = df_eval_pred.groupby("StudyInstanceUID")["Slice"].max().to_dict()
vert_cols = [f'C{i}_effnet_vert' for i in range(1, 8)]

paths = Path("/home/jumpei.uchida/develop/data/rsna/train_images")
path_list = joblib.Parallel(n_jobs=-1)([
    joblib.delayed(make_path_list)(path,i)
    for i,path in tqdm(enumerate(list(paths.iterdir())))])
path_list.sort(key=lambda x: x[1])
path_list = [t[0] for t in path_list]
flag_df = pd.DataFrame(path_list,columns = ["StudyInstanceUID","is_reversed"])

2019it [00:00, 229344.05it/s]


In [39]:

images = joblib.Parallel(n_jobs=-1)([
    joblib.delayed(resize_origin)(uid,i,flag_df)
    for i,uid in tqdm(enumerate(list(uid_to_slice_map.keys())))])

images.sort(key=lambda x: x[1])
images = [t[0] for t in images]

470it [00:00, 223734.30it/s]


In [40]:
df_pred = pd.concat(images)

In [41]:
df_pred.head()

,C1_effnet_vert,C2_effnet_vert,C3_effnet_vert,C4_effnet_vert,C5_effnet_vert,C6_effnet_vert,C7_effnet_vert,StudyInstanceUID,Slice
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.2.826.0.1.3680043.10032,321
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.2.826.0.1.3680043.10032,320
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.2.826.0.1.3680043.10032,319
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.2.826.0.1.3680043.10032,318
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.2.826.0.1.3680043.10032,317


In [42]:
df_predict = df_eval_pred.merge(df_pred,on = ["StudyInstanceUID","Slice"],how ="left")
df_predict[vert_cols] += 0.000000000000000001
for cols in vert_cols:
    df_predict[cols] = df_predict[cols].values * df_predict["pred"].values
df_predict = df_predict.drop("patient_overall",axis = 1).groupby(["StudyInstanceUID","split"]).max().reset_index()
df_predict = df_predict[['StudyInstanceUID', 'split','pred',
       'C1_effnet_vert', 'C2_effnet_vert', 'C3_effnet_vert', 'C4_effnet_vert',
       'C5_effnet_vert', 'C6_effnet_vert', 'C7_effnet_vert']]


In [43]:

df_train  = df_train.drop_duplicates("StudyInstanceUID")

In [44]:
df_predict = df_predict.merge(df_train.drop("split",axis = 1),on = ["StudyInstanceUID"],how ="left")

In [45]:
df_predict

,StudyInstanceUID,split,pred,C1_effnet_vert,C2_effnet_vert,C3_effnet_vert,C4_effnet_vert,C5_effnet_vert,C6_effnet_vert,C7_effnet_vert,...,C6,C7,patient_overall,C1_fracture,C2_fracture,C3_fracture,C4_fracture,C5_fracture,C6_fracture,C7_fracture
0,1.2.826.0.1.3680043.10032,0.0,0.1478,0.091245,0.142343,0.108804,0.095299,0.072745,0.096714,0.069381,...,0,0,0,0,0,0,0,0,0,0
1,1.2.826.0.1.3680043.10051,1.0,0.2568,0.200183,0.190219,0.148100,0.067930,0.080379,0.025422,0.071985,...,0,0,1,0,0,0,1,0,0,0
2,1.2.826.0.1.3680043.10062,2.0,0.1716,0.158870,0.134500,0.098955,0.097400,0.058894,0.019059,0.029292,...,0,0,0,0,0,0,0,0,0,0
3,1.2.826.0.1.3680043.10136,2.0,0.3100,0.309000,0.300184,0.277949,0.286900,0.114430,0.044750,0.039725,...,0,0,0,0,0,0,0,0,0,0
4,1.2.826.0.1.3680043.10579,2.0,0.3440,0.173635,0.331415,0.270658,0.185075,0.198500,0.143536,0.117745,...,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,1.2.826.0.1.3680043.9443,1.0,0.1897,0.178021,0.132722,0.146100,0.140290,0.087085,0.040000,0.010990,...,0,0,0,0,0,0,0,0,0,0
466,1.2.826.0.1.3680043.9447,1.0,0.1636,0.069823,0.063600,0.067100,0.094500,0.163600,0.134961,0.142700,...,0,0,1,0,1,0,0,0,0,0
467,1.2.826.0.1.3680043.9809,1.0,0.1512,0.124396,0.151200,0.147684,0.072400,0.013686,0.019673,0.009930,...,0,0,0,0,0,0,0,0,0,0
468,1.2.826.0.1.3680043.9926,2.0,0.1290,0.116595,0.119930,0.111855,0.089972,0.062087,0.060763,0.092350,...,0,0,1,0,1,0,0,0,0,0


In [46]:
target_cols = ["patient_overall"]+[f"C{i}_fracture" for i in range(1,7+1)]
pred_cols = ["pred"]+[f"C{i}_effnet_vert" for i in range(1,7+1)]
valid_list = []
DEVICE ="cuda"
for fold in range(N_FOLDS):
    df_temp  = df_predict.query("split == @fold")
    eval_targets = df_temp[target_cols].values
    eval_predictions = np.stack(df_temp[pred_cols].values.tolist())
    valid_score = custom_metric.weighted_loss(torch.logit(torch.as_tensor(eval_predictions)).to(DEVICE).to(torch.float), torch.as_tensor(eval_targets).to(DEVICE).to(torch.float))
    valid_list.append(valid_score.cpu())
    print(f'Valid_CV score Fold_{fold}:', valid_score)

Valid_CV score Fold_0: tensor(0.7476, device='cuda:0')
Valid_CV score Fold_1: tensor(0.8786, device='cuda:0')
Valid_CV score Fold_2: tensor(0.7613, device='cuda:0')


In [47]:
np.mean(valid_list)

0.795852

In [48]:
df_predictss = df_predict[['StudyInstanceUID', 'split', 'pred', 'C1_effnet_vert', 'C2_effnet_vert',
       'C3_effnet_vert', 'C4_effnet_vert', 'C5_effnet_vert', 'C6_effnet_vert',
       'C7_effnet_vert']].rename(columns ={f"C{i}_effnet_vert":f"C{i}" for i in range(1,7+1)})
df_predictss = df_predictss.rename(columns = {"pred":"overall","split":"fold"})

In [49]:
df_predictss.to_csv("binary_effnetv2_unet_pred.csv",index = False)